In [ ]:
from pathlib import Path
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data_ws = "/content/drive/MyDrive/2025 雙北黑客松/data/防災應變-旅遊景點周圍防災設施分析/"
assert 'readme.md' in os.listdir(data_ws)
os.chdir(data_ws)

In [ ]:
aed = pd.read_csv(data_ws + 'clean/合併/' + 'AED_list.csv')
aed.head()

,場所名稱,場所縣市,場所區域,場所地址,場所分類,場所類型,地點LAT,地點LNG
0,一滴水紀念館旅客服務中心,新北市,淡水區,新北市淡水區中正路一段6巷30號,觀光旅遊地區,文化園區,25.178206,121.430772
1,國立臺灣師範大學校本部(和平東路1段162號),臺北市,大安區,臺北市大安區和平東路1段162號,學校、大型集會場所,大專院校,25.025797,121.526706
2,新北市永和區秀朗國民小學,新北市,永和區,新北市永和區得和路202號,學校、大型集會場所,國小,24.998238,121.522580
3,財政部北區國稅局板橋分局,新北市,板橋區,新北市板橋區文化路一段48號,其他,其他場所,25.011976,121.459570
4,國立故宮博物院,臺北市,士林區,臺北市士林區至善路2段221號,大型休閒場所,博物館,25.102350,121.548432


In [ ]:
population = pd.read_csv(data_ws + 'clean/合併/' + 'merged_population.csv')
population = population.sort_values("鄉鎮市區名稱")
population.head()

,縣市代碼,縣市名稱,鄉鎮市區代碼,鄉鎮市區名稱,戶數,人口數,男性人口數,女性人口數,資料時間
131,65000,新北市,65000090,三峽區,40345,111588,56280,55308,103Y12M
90,65000,新北市,65000090,三峽區,39459,110010,55635,54375,102Y12M
167,65000,新北市,65000090,三峽區,41141,112708,56701,56007,104Y12M
459,65000,新北市,65000090,三峽區,45068,115274,57525,57749,111Y12M
49,65000,新北市,65000090,三峽區,38502,107794,54662,53132,101Y12M


In [ ]:
# Convert '資料時間' to a comparable format
population['資料時間_year'] = population['資料時間'].str.extract(r'(\d+)Y').astype(int)

# Find the latest year for each location
latest_year_per_location = population.groupby('鄉鎮市區名稱')['資料時間_year'].max().reset_index()

# Merge to get the rows corresponding to the latest year for each location
population_latest_year = pd.merge(population, latest_year_per_location, on=['鄉鎮市區名稱', '資料時間_year'], how='inner')
population = population_latest_year.sort_values("鄉鎮市區名稱")

population.head()

,縣市代碼,縣市名稱,鄉鎮市區代碼,鄉鎮市區名稱,戶數,人口數,男性人口數,女性人口數,資料時間,資料時間_year
0,65000,新北市,65000090,三峽區,46754,115335,57359,57976,113Y12M,113
1,65000,新北市,65000210,三芝區,10090,22080,11349,10731,113Y12M,113
2,65000,新北市,65000020,三重區,165752,383355,187023,196332,113Y12M,113
3,65000,新北市,65000030,中和區,177153,405956,194887,211069,113Y12M,113
4,63000,臺北市,63000040,中山區,101946,215245,98555,116690,113Y12M,113


In [ ]:
population['縣市名稱'].unique(), aed['場所縣市'].unique()

(array(['新北市', '臺北市'], dtype=object), array(['新北市', '臺北市'], dtype=object))

# Task
Calculate the total count, per capita count, and spatial dispersion of AEDs for each administrative region based on the data in "aed.csv" and "population.csv".

## Calculate AED count per region

### Subtask:
Group the `aed` DataFrame by administrative region (`場所縣市` and `場所區域`) and count the number of AEDs in each region.


**Reasoning**:
Group the aed DataFrame by administrative region and count the number of AEDs in each region.



In [ ]:
aed_count_by_region = aed.groupby(['場所縣市', '場所區域'])['場所名稱'].count().reset_index(name='aed_count')
aed_count_by_region.head()

,場所縣市,場所區域,aed_count
0,新北市,三峽區,64
1,新北市,三芝區,27
2,新北市,三重區,119
3,新北市,中和區,114
4,新北市,五股區,39


In [ ]:
aed_count_by_region.sample()

,場所縣市,場所區域,aed_count
33,臺北市,北投區,183


In [ ]:
# aed_count_by_region.to_csv(data_ws + 'component_ready/' + 'aed_count_by_region.csv', index=False)

aed_count_by_region.to_csv(data_ws + 'component_ready/' + 'aed_count_by_region.csv', index=False)

aed_count_by_region_tp = aed_count_by_region[aed_count_by_region['場所縣市'] == '臺北市']
aed_count_by_region_tp.to_csv(data_ws + 'component_ready/' + 'aed_count_by_region_tp.csv', index=False)


In [ ]:
aed_count_by_region_tp

,場所縣市,場所區域,aed_count
29,臺北市,中山區,306
30,臺北市,中正區,274
31,臺北市,信義區,267
32,臺北市,內湖區,204
33,臺北市,北投區,183
34,臺北市,南港區,169
35,臺北市,士林區,230
36,臺北市,大同區,107
37,臺北市,大安區,327
38,臺北市,文山區,242


## Calculate AED per capita

### Subtask:
Merge the AED count data with the `population` DataFrame to get the population for each region.

Then, calculate AED per capita by dividing the AED count by the population.


**Reasoning**:
Merge the AED count data with the population DataFrame and calculate AED per capita.



In [ ]:
population.sort_values("鄉鎮市區名稱").head()

,縣市代碼,縣市名稱,鄉鎮市區代碼,鄉鎮市區名稱,戶數,人口數,男性人口數,女性人口數,資料時間,資料時間_year
0,65000,新北市,65000090,三峽區,46754,115335,57359,57976,113Y12M,113
1,65000,新北市,65000210,三芝區,10090,22080,11349,10731,113Y12M,113
2,65000,新北市,65000020,三重區,165752,383355,187023,196332,113Y12M,113
3,65000,新北市,65000030,中和區,177153,405956,194887,211069,113Y12M,113
4,63000,臺北市,63000040,中山區,101946,215245,98555,116690,113Y12M,113


In [ ]:
# group by region from population first, then able to merge
population.groupby(['縣市名稱', '鄉鎮市區名稱'])['人口數'].sum().reset_index().head()

,縣市名稱,鄉鎮市區名稱,人口數
0,新北市,三峽區,115335
1,新北市,三芝區,22080
2,新北市,三重區,383355
3,新北市,中和區,405956
4,新北市,五股區,93860


In [ ]:
merged_df = pd.merge(
    aed_count_by_region,
    population,
    left_on=['場所縣市', '場所區域'],
    right_on=['縣市名稱', '鄉鎮市區名稱'],
    how='left',
)
# merged_df['aed_per_capita'] = merged_df['aed_count'] / merged_df['人口數']
# merged_df.fillna(0, inplace=True) # Handle potential missing values by filling with 0
# merged_df.head()

In [ ]:
merged_df['people_per_aed'] = merged_df['人口數'] / merged_df['aed_count']
merged_df.fillna(0, inplace=True) # Handle potential missing values (regions with 0 AEDs)
merged_df.head()

,場所縣市,場所區域,aed_count,縣市代碼,縣市名稱,鄉鎮市區代碼,鄉鎮市區名稱,戶數,人口數,男性人口數,女性人口數,資料時間,資料時間_year,people_per_aed
0,新北市,三峽區,64,65000,新北市,65000090,三峽區,46754,115335,57359,57976,113Y12M,113,1802.109375
1,新北市,三芝區,27,65000,新北市,65000210,三芝區,10090,22080,11349,10731,113Y12M,113,817.777778
2,新北市,三重區,119,65000,新北市,65000020,三重區,165752,383355,187023,196332,113Y12M,113,3221.470588
3,新北市,中和區,114,65000,新北市,65000030,中和區,177153,405956,194887,211069,113Y12M,113,3561.017544
4,新北市,五股區,39,65000,新北市,65000150,五股區,39187,93860,46641,47219,113Y12M,113,2406.666667


## Calculate spatial dispersion of AED

### Subtask:
Calculate the spatial dispersion of AEDs within each region using the provided latitude and longitude data.


**Reasoning**:
Calculate the standard deviation of latitude and longitude for each region and sum them to represent spatial dispersion.



In [ ]:
spatial_dispersion_df = aed.groupby(['場所縣市', '場所區域'])[['地點LAT', '地點LNG']].std().reset_index()
spatial_dispersion_df['spatial_dispersion'] = spatial_dispersion_df['地點LAT'] + spatial_dispersion_df['地點LNG']
spatial_dispersion_df.rename(columns={'地點LAT': 'lat_std', '地點LNG': 'lng_std'}, inplace=True)
spatial_dispersion_df.head()

,場所縣市,場所區域,lat_std,lng_std,spatial_dispersion
0,新北市,三峽區,0.027553,0.023056,0.050610
1,新北市,三芝區,0.025615,0.012379,0.037994
2,新北市,三重區,0.010786,0.008613,0.019399
3,新北市,中和區,0.008747,0.014430,0.023176
4,新北市,五股區,0.024467,0.013095,0.037562


In [ ]:
# prompt: If your goal is to produce a single “spatial dispersion” number that roughly equals “how far (on average) AEDs are spread,” you can:
# 	1.	Convert each lat/lng into a projected coordinate or a “geodesic distance” so that one unit = one meter (or one kilometer).
# 	2.	Compute a true 2D dispersion metric (for example, the standard deviation of the distance of each AED to the region’s centroid).


from shapely.geometry import Point
import geopandas as gpd
from pyproj import CRS
from pyproj import Transformer

# Define the original CRS (WGS84) and the target CRS (a projected CRS, e.g., TWD97 / TM2 zone north - epsg:3826)
# Note: TWD97 is appropriate for Taiwan. Use a different CRS if the data is elsewhere.
# Find appropriate CRS here: https://epsg.io/
transformer = Transformer.from_crs(CRS("epsg:4326"), CRS("epsg:3826"), always_xy=True)

def calculate_spatial_dispersion_projected(df):
    # Convert lat/lng to Point objects and project to a suitable CRS
    geometry = [Point(xy) for xy in zip(df['地點LNG'], df['地點LAT'])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="epsg:4326")

    # Apply the transformation
    gdf['geometry_projected'] = gdf['geometry'].apply(lambda point: Point(transformer.transform(point.x, point.y)))

    # Group by region and calculate the centroid of the projected points
    region_centroids = gdf.groupby(['場所縣市', '場所區域'])['geometry_projected'].apply(lambda points: points.unary_union.centroid)

    # Calculate the distance of each point to its region's centroid and then the standard deviation of these distances
    spatial_dispersion = {}
    for (county, region), centroid in region_centroids.items():
        region_aeds_projected = gdf[(gdf['場所縣市'] == county) & (gdf['場所區域'] == region)]['geometry_projected']
        distances = region_aeds_projected.apply(lambda point: point.distance(centroid))
        spatial_dispersion[(county, region)] = distances.std()

    # Convert the dictionary to a DataFrame
    spatial_dispersion_df_projected = pd.DataFrame.from_dict(spatial_dispersion, orient='index', columns=['spatial_dispersion_projected']).reset_index()
    spatial_dispersion_df_projected[['場所縣市', '場所區域']] = pd.DataFrame(spatial_dispersion_df_projected['index'].tolist(), index=spatial_dispersion_df_projected.index)
    spatial_dispersion_df_projected.drop(columns=['index'], inplace=True)

    return spatial_dispersion_df_projected

# Calculate spatial dispersion using projected coordinates
spatial_dispersion_projected_df = calculate_spatial_dispersion_projected(aed)
spatial_dispersion_projected_df.head()

<ipython-input-16-dd2b9c43606e>:25: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  region_centroids = gdf.groupby(['場所縣市', '場所區域'])['geometry_projected'].apply(lambda points: points.unary_union.centroid)


,spatial_dispersion_projected,場所縣市,場所區域
0,2278.363139,新北市,三峽區
1,1819.845751,新北市,三芝區
2,667.121768,新北市,三重區
3,842.279231,新北市,中和區
4,1427.989269,新北市,五股區


In [ ]:
# Merge the spatial dispersion data with the combined count and per capita data
final_merged_df = pd.merge(
    merged_df,
    spatial_dispersion_projected_df,
    on=['場所縣市', '場所區域'],
    how='left',
)

final_merged_df.fillna(0, inplace=True) # Fill missing dispersion values with 0
final_merged_df.head()

,場所縣市,場所區域,aed_count,縣市代碼,縣市名稱,鄉鎮市區代碼,鄉鎮市區名稱,戶數,人口數,男性人口數,女性人口數,資料時間,資料時間_year,people_per_aed,spatial_dispersion_projected
0,新北市,三峽區,64,65000,新北市,65000090,三峽區,46754,115335,57359,57976,113Y12M,113,1802.109375,2278.363139
1,新北市,三芝區,27,65000,新北市,65000210,三芝區,10090,22080,11349,10731,113Y12M,113,817.777778,1819.845751
2,新北市,三重區,119,65000,新北市,65000020,三重區,165752,383355,187023,196332,113Y12M,113,3221.470588,667.121768
3,新北市,中和區,114,65000,新北市,65000030,中和區,177153,405956,194887,211069,113Y12M,113,3561.017544,842.279231
4,新北市,五股區,39,65000,新北市,65000150,五股區,39187,93860,46641,47219,113Y12M,113,2406.666667,1427.989269


In [ ]:
# Display the relevant columns for the final result
final_result = final_merged_df[['場所縣市', '場所區域', 'aed_count', '人口數', 'people_per_aed', 'spatial_dispersion_projected']]
final_result['people_per_aed'] = final_result['people_per_aed'].astype(int)
final_result['spatial_dispersion_projected'] = final_result['spatial_dispersion_projected'].astype(int)
final_result.head()

<ipython-input-18-3ff8a5bfee0f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_result['people_per_aed'] = final_result['people_per_aed'].astype(int)
<ipython-input-18-3ff8a5bfee0f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_result['spatial_dispersion_projected'] = final_result['spatial_dispersion_projected'].astype(int)


,場所縣市,場所區域,aed_count,人口數,people_per_aed,spatial_dispersion_projected
0,新北市,三峽區,64,115335,1802,2278
1,新北市,三芝區,27,22080,817,1819
2,新北市,三重區,119,383355,3221,667
3,新北市,中和區,114,405956,3561,842
4,新北市,五股區,39,93860,2406,1427


In [ ]:
# final_result.to_csv(data_ws + 'component_ready/' + 'people_per_aed_spatial_dispersion.csv', index=False)

final_result.to_csv(data_ws + 'component_ready/' + 'people_per_aed_spatial_dispersion.csv', index=False)

final_result_tp = final_result[final_result['場所縣市'] == '臺北市']
final_result_tp.to_csv(data_ws + 'component_ready/' + 'people_per_aed_spatial_dispersion_tp.csv', index=False)

In [ ]:
final_result[final_result['場所縣市'] == "新北市"]

,場所縣市,場所區域,aed_count,人口數,people_per_aed,spatial_dispersion_projected
0,新北市,三峽區,64,115335,1802,2278
1,新北市,三芝區,27,22080,817,1819
2,新北市,三重區,119,383355,3221,667
3,新北市,中和區,114,405956,3561,842
4,新北市,五股區,39,93860,2406,1427
5,新北市,八里區,42,42674,1016,1490
6,新北市,土城區,51,241224,4729,1711
7,新北市,坪林區,11,6324,574,2316
8,新北市,平溪區,13,4049,311,1276
9,新北市,新店區,119,306571,2576,1900


In [ ]:
final_result[final_result['場所縣市'] == "臺北市"]

,場所縣市,場所區域,aed_count,人口數,people_per_aed,spatial_dispersion_projected
29,臺北市,中山區,306,215245,703,808
30,臺北市,中正區,274,148375,541,14887
31,臺北市,信義區,267,205067,768,718
32,臺北市,內湖區,204,273748,1341,798
33,臺北市,北投區,183,240623,1314,2249
34,臺北市,南港區,169,112643,666,16756
35,臺北市,士林區,230,264194,1148,1648
36,臺北市,大同區,107,118992,1112,650
37,臺北市,大安區,327,289908,886,470
38,臺北市,文山區,242,258333,1067,807


## Combine results

### Subtask:
Merge all calculated metrics into a single DataFrame.


**Reasoning**:
Merge the `merged_df` and `spatial_dispersion_df` DataFrames using '場所縣市' and '場所區域' as the merge keys with a left merge.



In [ ]:
final_merged_df = pd.merge(
    merged_df,
    spatial_dispersion_df,
    on=['場所縣市', '場所區域'],
    how='left'
)
display(final_merged_df.head())

,場所縣市,場所區域,aed_count,縣市代碼,縣市名稱,鄉鎮市區代碼,鄉鎮市區名稱,戶數,人口數,男性人口數,女性人口數,資料時間,資料時間_year,people_per_aed,lat_std,lng_std,spatial_dispersion
0,新北市,三峽區,64,65000,新北市,65000090,三峽區,46754,115335,57359,57976,113Y12M,113,1802.109375,0.027553,0.023056,0.050610
1,新北市,三芝區,27,65000,新北市,65000210,三芝區,10090,22080,11349,10731,113Y12M,113,817.777778,0.025615,0.012379,0.037994
2,新北市,三重區,119,65000,新北市,65000020,三重區,165752,383355,187023,196332,113Y12M,113,3221.470588,0.010786,0.008613,0.019399
3,新北市,中和區,114,65000,新北市,65000030,中和區,177153,405956,194887,211069,113Y12M,113,3561.017544,0.008747,0.014430,0.023176
4,新北市,五股區,39,65000,新北市,65000150,五股區,39187,93860,46641,47219,113Y12M,113,2406.666667,0.024467,0.013095,0.037562


## 各個鄉鎮中可以被用作發生水災、震災、土石流、海嘯時的避難地點數量

In [ ]:
df = pd.read_csv(data_ws + 'clean/合併/' + '避難收容處所合併.csv')
to_exclude = ['收容所編號', '名稱', '門牌地址', '類型', '聯絡人姓名', '聯絡人連絡電話', '管理人姓名' ,'管理人連絡電話', '備考', '完整地址', '服務里別']
df.drop(columns=to_exclude, inplace=True)
df.head()

,縣市,鄉鎮,村里,水災,震災,土石流,海嘯,救濟支站,無障礙設施,室內,室外,容納人數,收容所面積（平方公尺）,longitude,latitude
0,臺北市,中正區,林興里,Y,備用,N,N,Y,Y,Y,N,52,209,121.527516,25.019244
1,臺北市,中正區,黎明里,Y,Y,N,N,Y,Y,Y,N,157,1062,121.516630,25.045066
2,臺北市,中正區,建國里,Y,Y,N,N,Y,Y,Y,N,130,520,121.515291,25.037545
3,臺北市,中正區,黎明里,Y,Y,N,N,N,Y,N,Y,5424,21696,121.515585,25.040278
4,臺北市,中正區,東門里,Y,備用,N,N,Y,N,Y,N,250,2457.96,121.519306,25.038368


In [ ]:
# 1. 定義災別欄位
disaster_cols = ["水災", "震災", "土石流", "海嘯"]

# 2. 將「Y」或「是」視為 True，其餘視為 False，並存到新欄位 (col + "_bool")
for col in disaster_cols:
    df[col + "_bool"] = df[col].isin(["Y", "是"])

# 3. 針對「鄉鎮」分組，把各災別的布林 (True/False) 做 sum()
#    因為 True 當作 1、False 當作 0，sum() 就能得到「該鄉鎮底下，有多少個地點可以對應該災別」
grouped = (
    df
    .groupby(["縣市", "鄉鎮"])[ [col + "_bool" for col in disaster_cols] ]
    .sum()
    .reset_index()
)

# 4. 重命名欄位為較易閱讀的名稱
grouped = grouped.rename(columns={
    "水災_bool": "flood_shelter_count",
    "震災_bool": "earthquake_shelter_count",
    "土石流_bool": "landslide_shelter_count",
    "海嘯_bool": "tsunami_shelter_count",
    "縣市": "city"
})

# 5. （選項）如果要計算「該地點至少支援一種災別」的總數，可以再額外做：
#    先為每一列新增「任一災別」欄：只要該列任一災別欄位是 "Y" 或 "是"，就算 True
df["任一災別"] = df[disaster_cols].apply(lambda row: row.isin(["Y", "是"]).any(), axis=1)

#    再按「鄉鎮」分組加總
total_any = (
    df
    .groupby("鄉鎮")["任一災別"]
    .sum()
    .reset_index()
    .rename(columns={"任一災別": "任一災別避難地點數"})
)

# 6. 合併「各災別避難地點數」與「任一災別避難地點數」
result = pd.merge(grouped, total_any, on="鄉鎮", how="left")

# 7. 顯示最終結果
result.head()

,city,鄉鎮,flood_shelter_count,earthquake_shelter_count,landslide_shelter_count,tsunami_shelter_count,任一災別避難地點數
0,新北市,三峽區,20,21,21,21,21
1,新北市,三芝區,9,9,9,5,9
2,新北市,三重區,25,35,35,35,35
3,新北市,中和區,24,23,24,24,24
4,新北市,五股區,13,7,12,13,13


In [ ]:
result.to_csv(data_ws + 'component_ready/' + 'disaster.csv', index=False)

result_tp = result[result['city'] == '臺北市']
result_tp.to_csv(data_ws + 'component_ready/' + 'disaster_tp.csv', index=False)